In [19]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", 200)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from sklearn.cluster import DBSCAN

print("Import complete")

Import complete


### Import all the datasets

In [20]:
# test data
test = pd.read_csv(r"../files/test_set_features.csv", index_col="respondent_id")

# train data
train = pd.read_csv(r"../files/training_set_features.csv", index_col="respondent_id")

# target
training_labels = pd.read_csv(r"../files/training_set_labels.csv", index_col="respondent_id")
label_h1n1 = training_labels[["h1n1_vaccine"]]

# check whether rows are equal
print("train df => rows: %s, cols: %s" % (train.shape[0], train.shape[1]))
print("train labels df => rows: %s, cols: %s" % (training_labels.shape[0], training_labels.shape[1]))
print("test df => rows: %s, cols: %s" % (test.shape[0], test.shape[1]))

assert(train.shape[1] == test.shape[1])

train df => rows: 26707, cols: 35
train labels df => rows: 26707, cols: 2
test df => rows: 26708, cols: 35


### Check for duplicates

In [21]:
# check for duplicates
original_data_dup_count = np.sum(train.duplicated())
original_test_dup_count = np.sum(test.duplicated())

print("duplicates in original training dataset: %s" % original_data_dup_count)
print("duplicates in original testing dataset: %s" % original_test_dup_count)

assert(original_data_dup_count == 0 and original_test_dup_count == 0)

duplicates in original training dataset: 0
duplicates in original testing dataset: 0


In [22]:
# displaying object column data
# both `train` and `test` have same columns

df = train
col_details = []
for col in df.columns:
    col_details.append((col, df[col].dtype, df[col].nunique(), list(df[col].unique()), df[col].isnull().sum()))
col_details.sort(key=lambda x: 1 / x[-3])

temp = pd.DataFrame(col_details, columns=["Column", "Dtype", "N_Unique", "Unique_vals", "Null values"])
temp

,Column,Dtype,N_Unique,Unique_vals,Null values
0,employment_occupation,object,23,"[nan, xgwztkwe, xtkaffoo, emcorrxb, vlluhbov, ...",13470
1,employment_industry,object,21,"[nan, pxcmvdjn, rucpziij, wxleyezf, saaquncn, ...",13330
2,hhs_geo_region,object,10,"[oxchjgsf, bhuqouqj, qufhixun, lrircsnp, atmpe...",0
3,opinion_h1n1_vacc_effective,float64,5,"[3.0, 5.0, 4.0, 2.0, 1.0, nan]",391
4,opinion_h1n1_risk,float64,5,"[1.0, 4.0, 3.0, 2.0, 5.0, nan]",388
5,opinion_h1n1_sick_from_vacc,float64,5,"[2.0, 4.0, 1.0, 5.0, 3.0, nan]",395
6,opinion_seas_vacc_effective,float64,5,"[2.0, 4.0, 5.0, 3.0, 1.0, nan]",462
7,opinion_seas_risk,float64,5,"[1.0, 2.0, 4.0, 3.0, 5.0, nan]",514
8,opinion_seas_sick_from_vacc,float64,5,"[2.0, 4.0, 1.0, 5.0, nan, 3.0]",537
9,age_group,object,5,"[55 - 64 Years, 35 - 44 Years, 18 - 34 Years, ...",0


### Identifying categorical columns and numerical columns

In [23]:
categorical_columns = np.where(train.dtypes == object)[0]
numerical_columns =  np.where(train.dtypes != object)[0]

assert (len(categorical_columns) + len(numerical_columns) == len(train.columns))

### Filling missing values

In [24]:
# fill numeric values with its mean
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy = 'mean'))
])

non_numeric_preprocessing_steps = Pipeline([
     ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot_encoder', OneHotEncoder())
])


preprocessor = ColumnTransformer(
    transformers = [
        ('numeric', numeric_preprocessing_steps, numerical_columns),
        ('non_numeric', non_numeric_preprocessing_steps, categorical_columns)
    ],
    remainder = "drop"
)

In [25]:
# transform whole dataset
train = pd.DataFrame(preprocessor.fit_transform(train), index=train.index)
test = pd.DataFrame(preprocessor.transform(test), index=test.index)

### Correlation analysis

In [26]:
# setting threshold
threshold = 0.8

corr = train.corr().abs()

# select upper triangle of correlations
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

removed_cols = [column for column in upper.columns if any(upper[column] > threshold)]

removed_cols

[38, 44, 45, 47, 50, 52, 75, 91, 98]

### Prune columns which have above threshold co-relation

In [27]:
train = train.drop(columns = removed_cols)
test = test.drop(columns = removed_cols)

### differentiate train and test data

In [28]:
train["type"] = "train"
test["type"] = "test"

# create a data column by merging both train and label set
data = pd.concat([train, test], ignore_index=False)

data.tail(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,type
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53413,1.517658,-0.424715,-0.22661,0.614936,-0.272201,0.459586,-0.747788,1.401639,-1.448623,-0.531568,-0.701389,-0.628655,3.332873,-0.354995,6.547647e-17,-1.836999,0.511417,1.205168,-0.023916,0.202767,-0.088611,0.150650,-0.575963,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
53414,0.419111,-0.424715,-0.22661,-1.626185,-0.272201,0.459586,-0.747788,-0.713450,0.690311,1.881227,-0.701389,-0.628655,-0.300041,-0.354995,3.697637e-01,1.140915,-1.044380,-0.262463,0.896433,0.924774,-0.088611,-1.176654,-0.575963,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,test


In [29]:
# now that data is ready to train, publish it as a csv file
data.to_csv("../files/for_train.csv", index=True)
print("success")


success
